In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import sys
import sys
import auto_encoders 
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import cv2

In [2]:
import os
train_data_loadpath='/mnt/sda/sunche/datasets/Anomaly/Avenue/train/'
a=os.listdir(train_data_loadpath)
print(len(os.listdir(train_data_loadpath)))
m=range(10)
print(str(m[3])+'.jpg')

15332
3.jpg


In [3]:
BATCH_SIZE=64
train_data_loadpath='/mnt/sda/sunche/datasets/Anomaly/Avenue/train/'
train_label_loadpath='/mnt/sda/sunche/datasets/Anomaly/Avenue/train/'
root='/mnt/sda/sunche/datasets/Anomaly/Avenue/'
def Gaussian(X,rate=0.05):
    # Salt and pepper noise
    drop = np.random.normal(0,rate, X.shape)
    X=X+drop
    return X
class MyDataset(Dataset):
    def __init__(self, train_data_loadpath,transform=None, target_transform=None,):
        self.path=train_data_loadpath
        self.files=os.listdir(train_data_loadpath)
        self.inds=range(len(self.files))
    def __getitem__(self, index):
        img_path=os.path.join(self.path,str(self.inds[index]+1)+'.jpg')
        img = cv2.resize(cv2.imread(img_path),(224,224))
        img_npy=np.array(img).astype(np.float32)/255.0
        
        img_npy = Gaussian(img_npy)
        img_npy = np.reshape(img_npy,[224,224,3])
        img_npy = np.transpose(img_npy,[2,0,1])
        label=img_npy.copy()

        img_npy=torch.from_numpy(img_npy).float()
        label=torch.from_numpy(label).float()

        return img_npy,label

    def __len__(self):
        return len(self.files)
train_data=MyDataset(train_data_loadpath)  
torch.set_num_threads(1)
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE,num_workers=16, shuffle=True,pin_memory=False)  

In [4]:
epoch_=10
model = auto_encoders.AE_2D_Net_3layers_3()
# model = auto_encoders.ConvLSTM_Net_3layers3x3()
# model = auto_encoders.AE_3D_Net_6()
model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.cuda()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3,weight_decay=0)
loss_func = torch.nn.MSELoss()
loss_func.cuda() 
num=0
model.train()
d_steps=1
g_steps=1
record_loss=[]
for epoch in range(epoch_):
    print('epoch {}'.format(epoch + 1))
    # training-----------------------------
    train_loss = 0.
    for i,(batch_x, batch_y) in enumerate(train_loader):
      batch_x = batch_x.cuda()
      batch_y = batch_y.cuda()
#       print(batch_x.size())
      current_batch=list(batch_x.size())[0]
      for g_index in range(g_steps):
        model.zero_grad()
        
        out,_ = model(batch_x)
#         print(out.size())
        loss= loss_func(out.squeeze(1), batch_x)



        train_loss += loss.item()

        loss.backward()

        optimizer.step()
        print('epoch:{},batchID{},RLoss{:.6f}'.format(epoch,i,loss.item()))

    if epoch%10==9:
      state = {'epoch': epoch+1,
               'model_state': model.state_dict(),
               'optimizer_state' : optimizer.state_dict(),}
      torch.save(state, './savepath_lstm_Net_6/model'+str(epoch+1)+'.pkl')
      record_loss.append(train_loss)
    print('Train Loss: {:.6f}'.format(train_loss / (len(train_data))))
print(record_loss)

epoch 1
epoch:0,batchID0,RLoss1.995917
epoch:0,batchID1,RLoss1.875624
epoch:0,batchID2,RLoss1.458169
epoch:0,batchID3,RLoss0.860215
epoch:0,batchID4,RLoss0.476825
epoch:0,batchID5,RLoss0.400715
epoch:0,batchID6,RLoss0.366725
epoch:0,batchID7,RLoss0.307031
epoch:0,batchID8,RLoss0.240748
epoch:0,batchID9,RLoss0.213760
epoch:0,batchID10,RLoss0.229071
epoch:0,batchID11,RLoss0.231650
epoch:0,batchID12,RLoss0.215206
epoch:0,batchID13,RLoss0.186284
epoch:0,batchID14,RLoss0.162717
epoch:0,batchID15,RLoss0.153588
epoch:0,batchID16,RLoss0.154824
epoch:0,batchID17,RLoss0.156943
epoch:0,batchID18,RLoss0.152103
epoch:0,batchID19,RLoss0.143779
epoch:0,batchID20,RLoss0.130538
epoch:0,batchID21,RLoss0.124669
epoch:0,batchID22,RLoss0.121179
epoch:0,batchID23,RLoss0.120468
epoch:0,batchID24,RLoss0.117757
epoch:0,batchID25,RLoss0.113678
epoch:0,batchID26,RLoss0.108336
epoch:0,batchID27,RLoss0.103895
epoch:0,batchID28,RLoss0.098342
epoch:0,batchID29,RLoss0.098632
epoch:0,batchID30,RLoss0.096238
epoch:0,ba